In [5]:
import pandas as pd
import numpy as np
import math
import fiona
import geopandas
import json

from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter
from bokeh.palettes import brewer

from bokeh.io.doc import curdoc
from bokeh.models import Slider, HoverTool, Select
from bokeh.layouts import widgetbox, row, column

neighborhood_data = pd.read_csv('https://raw.githubusercontent.com/jerguy1928/tram_performance/master/tram_data.csv')

# Read the geojson map file for Realtor Neighborhoods into a GeoDataframe object
sf = geopandas.read_file('https://raw.githubusercontent.com/jerguy1928/tram_performance/master/Tram_Routes.geojson')

# Set the Coordinate Referance System (crs) for projections
# ESPG code 4283 is also referred to as GDA94 lat-long projection
sf.crs = {'init': 'epsg:4283'}

# Rename columns in geojson map file
sf = sf.rename(columns={'ROUTESHTNM': 'route','ROUTELONGN':'name', 'nid': 'subdist_no'}).set_geometry('geometry')

# Change neighborhood id (subdist_no) for correct code for Mount Davidson Manor and for parks
# sf.loc[sf['neighborhood_name'] == 'Mount Davidson Manor', 'subdist_no'] = '4n'
# sf.loc[sf['neighborhood_name'] == 'Golden Gate Park', 'subdist_no'] = '12a'
# sf.loc[sf['neighborhood_na'] == 'Presidio', 'subdist_no'] = '12b'
# sf.loc[sf['neighborhood_name'] == 'Lincoln Park', 'subdist_no'] = '12c'

# sf.sort_values(by=['subdist_no'])

# This dictionary contains the formatting for the data in the plots
format_data = [('route', 0, 100,'0,0', 'Route Number'),
               ('delivered_percent', 0, 100,'0%', 'Percent of services delivered'),
               ('ontime_percent',  0, 100,'0%', 'Percent on-time'),
              #  ('sf_mean', 500, 5000,'0,0', 'Average Square Footage'),
              #  ('price_sf_mean', 0, 2000,'$0,0', 'Average Price Per Square Foot'),
              #  ('min_income', 50000, 600000,'$0,0', 'Minimum Income Required')
              ]
 
#Create a DataFrame object from the dictionary 
format_df = pd.DataFrame(format_data, columns = ['field' , 'min_range', 'max_range' , 'format', 'verbage'])

# Create a function the returns json_data for the year selected by the user
def json_data(selectedYear):
    # yr = selectedYear
    
    # Pull selected year from neighborhood summary data
    # df_yr = neighborhood_data[neighborhood_data['year'] == yr]
    df_yr = neighborhood_data
    # Merge the GeoDataframe object (sf) with the neighborhood summary data (neighborhood)
    # merged = pd.merge(sf, df_yr, on='subdist_no', how='left')
    merged = pd.merge(sf, df_yr, on='route', how='left')
    
    
    # Fill the null values
   
    # values = {'year': yr, 'sale_price_count': 0, 'sale_price_mean': 0, 'sale_price_median': 0,
    #           'sf_mean': 0, 'price_sf_mean': 0, 'min_income': 0}
    # merged = merged.fillna(value=values)
    
    # Bokeh uses geojson formatting, representing geographical features, with json
    # Convert to json
    merged_json = json.loads(merged.to_json())
    
    # Convert to json preferred string-like object 
    json_data = json.dumps(merged_json)
    return json_data
  
# Define the callback function: update_plot
def update_plot(attr, old, new):
    # The input yr is the year selected from the slider
    yr = slider.value
    new_data = json_data(yr)
    
    # The input cr is the criteria selected from the select box
    cr = select.value
    input_field = format_df.loc[format_df['verbage'] == cr, 'field'].iloc[0]
    
    # Update the plot based on the changed inputs
    p = make_plot(input_field)
    
    # Update the layout, clear the old document and display the new document
    layout = column(p, widgetbox(select), widgetbox(slider))
    curdoc().clear()
    curdoc().add_root(layout)
    
    # Update the data
    geosource.geojson = new_data 
    
# Create a plotting function
def make_plot(field_name):    
  # Set the format of the colorbar
  min_range = format_df.loc[format_df['field'] == field_name, 'min_range'].iloc[0]
  max_range = format_df.loc[format_df['field'] == field_name, 'max_range'].iloc[0]
  field_format = format_df.loc[format_df['field'] == field_name, 'format'].iloc[0]

  # Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
  color_mapper = LinearColorMapper(palette = palette, low = min_range, high = max_range)

  # Create color bar.
  format_tick = NumeralTickFormatter(format=field_format)
  color_bar = ColorBar(color_mapper=color_mapper, label_standoff=18, formatter=format_tick,
  border_line_color=None, location = (0, 0))

  # Create figure object.
  verbage = format_df.loc[format_df['field'] == field_name, 'verbage'].iloc[0]

  p = figure(title = verbage + ' by Neighborhood for Single Family Homes in SF by Year - 2009 to 2018', 
             plot_height = 650, plot_width = 850,
             toolbar_location = None)
  p.xgrid.grid_line_color = None
  p.ygrid.grid_line_color = None
  p.axis.visible = False

  # Add patch renderer to figure. 
  p.patches('xs','ys', source = geosource, fill_color = {'field' : field_name, 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
  
  # Specify color bar layout.
  p.add_layout(color_bar, 'right')

  # Add the hover tool to the graph
  p.add_tools(hover)
  return p
    
    
# Input geojson source that contains features for plotting for:
# initial year 2018 and initial criteria sale_price_median
geosource = GeoJSONDataSource(geojson = json_data(2018))
# input_field = 'sale_price_median'
input_field = 'delivered_percent'


# Define a sequential multi-hue color palette.
palette = brewer['Blues'][8]

# Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

# Add hover tool
hover = HoverTool(tooltips = [ ('Percent of services delivered','$@delivered_percent{%}'),
                               ('Percent on-time', '$@ontime_percent{%}'),
                              #  ('Average Price', '$@sale_price_mean{,}'),
                              #  ('Median Price', '$@sale_price_median{,}'),
                              #  ('Average SF', '@sf_mean{,}'),
                              #  ('Price/SF ', '$@price_sf_mean{,}'),
                              #  ('Income Needed', '$@min_income{,}')
                               
                               ])


# format_data = [('route', 0, 100,'0,0', 'Route Number'),
#                ('delivered_percent', 0, 100,'0%', 'Percent of services delivered'),
#                ('ontime_percent',  0, 100,'0%', 'Percent on-time'),


# Call the plotting function
p = make_plot(input_field)

# Make a slider object: slider 
slider = Slider(title = 'Year',start = 2009, end = 2018, step = 1, value = 2018)
slider.on_change('value', update_plot)

# Make a selection object: select
select = Select(title='Select Criteria:', value='Median Sales Price', options=['Median Sales Price', 'Minimum Income Required',
                                                                               'Average Sales Price', 'Average Price Per Square Foot',
                                                                               'Average Square Footage', 'Number of Sales'])
select.on_change('value', update_plot)

# Make a column layout of widgetbox(slider) and plot, and add it to the current document
# Display the current document
layout = column(p, widgetbox(select), widgetbox(slider))
curdoc().add_root(layout)

# Use the following code to test in a notebook, comment out for transfer to live site
# Interactive features will not show in notebook
output_notebook()
show(p)

Loading BokehJS ...